In [1]:
#Vizualize demand spatial distribution in Amsterdam emoped pilot

In [2]:
### Imports 
import os
import math
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as patches
import json

import pandas as pd
import re
import xml.etree.ElementTree as ET

sys.path.append('../../')
from script.conversion.bison.coordinates import rd_to_utm
from mnms.graph.layers import PublicTransportLayer, MultiLayerGraph, OriginDestinationLayer
from mnms.generation.roads import generate_pt_line_road, generate_one_zone
from mnms.generation.layers import generate_bbox_origin_destination_layer
from mnms.vehicles.veh_type import Tram, Metro, Bus
from mnms.generation.zones import generate_one_zone
from mnms.mobility_service.public_transport import PublicTransportMobilityService
from mnms.time import TimeTable, Dt, Time
from mnms.io.graph import load_graph, save_graph, save_odlayer
from mnms.tools.render import draw_roads, draw_line, draw_odlayer
from mnms.tools.geometry import points_in_polygon, get_bounding_box


/Users/louis/opt/anaconda3/envs/mnms/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
### Parameters

# Files and directories
current_dir = os.getcwd()
indir = current_dir + '/inputs/'
outdir = current_dir + '/outputs/'
f = open('params.json')
params = json.load(f)

amsterdam_json_filepath = indir + params['fn_network'] # mlgraph with the road network only
ams_dmd_path = indir + params['fn_demand'] #'custom_demand_4-23.csv' # dmd for odlayer

In [4]:
### Get the MLGraph without TCs 
amsterdam_graph = load_graph(amsterdam_json_filepath)
roads = amsterdam_graph.roads

In [5]:
# Load dmd
db_dmd = pd.read_csv(ams_dmd_path, sep=';')

In [6]:
db_dmd

,ID,DEPARTURE,ORIGIN,DESTINATION
0,231818,16:00:00,634168.5276821428 5804137.687618688,630736.8116092718 5803553.228968799
1,115712,16:00:00,635289.9891833308 5798931.744649006,626252.8837445606 5801060.53731042
2,88312,16:00:00,630048.7947913781 5802558.532006365,625013.9409027101 5805957.982522872
3,133927,16:00:00,632137.9177222168 5797594.918169631,628288.8961136984 5809791.45440402
4,111865,16:00:00,632658.3877148335 5795254.896756441,632137.9177222168 5797594.918169631
...,...,...,...,...
288936,232048,17:59:59,634168.5276821428 5804137.687618688,633145.2705942673 5796474.513933667
288937,219546,17:59:59,629079.8491837555 5802316.11929759,631471.0008199599 5807574.539496145
288938,77969,17:59:59,633438.6326959758 5802309.617005215,635901.9828472091 5802651.519236161
288939,144960,17:59:59,633229.1213910822 5798142.114189578,625013.9409027101 5805957.982522872


In [7]:
nb_dmd = len(db_dmd)
origins = np.zeros((len(db_dmd),2))
destinations = np.zeros((len(db_dmd),2))
for i, row in db_dmd[:].iterrows():
    origins[i] = [float(o) for o in row['ORIGIN'].split(' ')]
    destinations[i] = [float(d) for d in row['DESTINATION'].split(' ')]

# OD points

In [8]:
origins_str = [str(p[0])+','+str(p[1]) for p in origins]
destinations_str = [str(p[0])+','+str(p[1]) for p in destinations]

In [9]:
points_str = np.unique(origins_str + destinations_str)
points = [[float(pp) for pp in p.split(',')] for p in points_str]

In [10]:
len(points)

935

In [11]:
# Save odlayer
odlayer = OriginDestinationLayer()

for i, od in enumerate(points):
    odlayer.create_origin_node(f"ORIGIN_{i}", od)
    odlayer.create_destination_node(f"DESTINATION_{i}", od)

In [12]:
odlayer.origins

{'ORIGIN_0': [618206.8426379834, 5807813.882732264],
 'ORIGIN_1': [618413.753228729, 5809781.9888599785],
 'ORIGIN_2': [618772.8192127077, 5808503.210018341],
 'ORIGIN_3': [619131.9533648283, 5807224.170893929],
 'ORIGIN_4': [619296.2265274503, 5808360.539122885],
 'ORIGIN_5': [619722.7602578658, 5802167.623113418],
 'ORIGIN_6': [619819.6719324354, 5808217.8577584885],
 'ORIGIN_7': [619850.4260722174, 5804665.311892333],
 'ORIGIN_8': [619929.6184065838, 5804137.260604206],
 'ORIGIN_9': [619972.1596257949, 5804969.711415222],
 'ORIGIN_10': [619983.8922566546, 5809354.130677865],
 'ORIGIN_11': [620057.2227236249, 5806634.356294085],
 'ORIGIN_12': [620167.2268408912, 5802552.864271971],
 'ORIGIN_13': [620246.4401255952, 5802024.65134739],
 'ORIGIN_14': [620410.7230337895, 5803161.803995203],
 'ORIGIN_15': [620422.3565051515, 5807547.347466694],
 'ORIGIN_16': [620453.2455516264, 5803994.378668912],
 'ORIGIN_17': [620617.489193579, 5805131.27254215],
 'ORIGIN_18': [620648.4030560774, 580157

In [13]:
save_odlayer(odlayer, indir + 'od_layer_od.json')